# Drawing figures relating to Logistic Map 

In [ ]:
from matplotlib import pyplot as plt
from matplotlib import patches as pts
import sys
sys.path.append("../lib")
from pylib import Point,DataFile
plt.rcParams['figure.subplot.bottom'] = 0.15
plt.rcParams['font.size']=20
plt.rcParams['font.family']='Times New Roman'
plt.rcParams['mathtext.fontset']='cm'
plt.rcParams['mathtext.default']='it'


In [ ]:
def logistic(x:float,l:float,n:int):
    """
    logistic map

    Parameters
    ------
    x: variable
    l: parameter lambda
    n: the number of iteration
    """
    y = 4 * l * x * (1 - x)
    if n == 1:
        return y
    return logistic(y, l, n-1)

In [ ]:
def drawTrajectory(filename:str,period=1):
    """
    drawing trajectory in (x_n,x_{n+1}) plane
    """
    data = DataFile(filename)
    l = data.getParam('lambda')
    ax = plt.subplot()
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_aspect('equal')
    ax.set_xlabel('$x_n$')
    ax.set_ylabel('$x_{n+1}$')
    #drawing simulation data
    xl,yl = data.readData()
    ax.plot(xl,yl)
    #drawing logistic map
    n = 512
    xl = [k/n for k in range(n)]
    yl = [logistic(x, l, period) for x in xl]
    ax.plot(xl, yl, c='black')

    ax.plot([0,1], [0,1], c='black')
    ax.text(.6, .1, f'$\\lambda={l}$')

    if period > 1:
        filename = filename.replace('.txt', '-period'+str(period)+'.txt')
    outputFilename = filename.replace('.txt', '.pdf')
    plt.savefig(outputFilename)

In [ ]:
def drawTrajectoryMag(filename:str,period=1,xmin=.47,w=.1,funcOnly=False):
    """
    drawing magnified trajectory in (x_n,x_{n+1}) plane
    """

    ax = plt.subplot()
    ax.set_xlim(xmin, xmin + w)
    ax.set_ylim(xmin, xmin + w)
    ax.set_xlabel('$x_n$')
    ax.set_ylabel('$x_{n+1}$')
    ax.set_aspect('equal')

    data = DataFile(filename)
    l = data.getParam('lambda')
    if not funcOnly:
        xl, yl = data.readData()
        ax.plot(xl, yl)

    n = 512
    xl = [k/n for k in range(n)]
    yl = [logistic(x, l, period) for x in xl]
    ax.plot(xl, yl, c='black')
    
    ax.plot([0,1], [0,1], c='black')
    if period > 1:
        filename = filename.replace('.txt', '-period'+str(period)+'.txt')
    outputFilename = filename.replace('.txt', '.pdf')
    plt.savefig(outputFilename)


In [ ]:
def drawTimeSequence(filename:str,tMin:int,tMax:int,chaos=False,showRange=False):
    """
    Drawing time sequence
    """
    ax = plt.subplot()
    ax.set_xlim(tMin, tMax)
    ax.set_ylim(0, 1)
    ax.set_xlabel('$t$')
    ax.set_ylabel('$x_{n}$')
    data = DataFile(filename)
    l = data.getParam('lambda')
    xl, yl=data.readData()
    ax.plot(xl, yl)
    if chaos:#draw another sequence with different initial value
        xl, yl=data.readData(y=2)
        ax.plot(xl, yl)
    ax.text(10, .1, f'$\\lambda={l}$')

    if showRange:
        #band structure
        ss = [logistic(.5,l,k) for k in range(1,5)]
        for s in ss:
            ax.plot([tMin, tMax],[s,s])
    
    outputFilename = filename.replace('.txt', '.pdf')
    plt.savefig(outputFilename)


In [ ]:
def overAllView(filename:str,fromLambda:float,withLine=False):
    """
    drawing bifurcation
    """
    ax = plt.subplot()
    ax.set_xlim(fromLambda, 1)
    ax.set_ylim(0, 1)
    ax.set_xlabel('$\\lambda$')
    ax.set_ylabel('$x_n$')

    xl,yl = DataFile(filename).readData()
    ax.scatter(xl, yl, s=.1)
    if withLine:
        n = 512
        lp=[Point(.9,.95), Point(.9,.2), Point(.9,.75), Point(.85,.55)]
        xl = [(1 - fromLambda) * x / n + fromLambda for x in range(n)]
        for k in range(4):
            yl = [logistic(.5, x, k+1) for x in xl]
            ax.plot(xl, yl, c='black')
            ax.text(lp[k].x, lp[k].y, str(k + 1))
        filename=filename.replace('.txt', '-WithLine.txt')
    outputFile = filename.replace('.txt', '.pdf')
    plt.savefig(outputFile)

In [ ]:
def drawDist(filename:str,h:float,recursion=0):
    """
    Drawing distribution
    """
    ax = plt.subplot()
    ax.set_xlim(0, 1)
    ax.set_ylim(0, h)

    data = DataFile(filename)
    l = data.getParam("lambda")
    n = int(data.getParam('numBin'))
    w = 1. / n
    xl, yl = data.readData()
    ax.bar(xl, yl, width=w)
    ax.text(.2, .8*h, f'$\\lambda={l}$')
    for k in range(recursion):
        x = logistic(.5,l,k+1)
        ax.plot([x,x],[0,2],c='r')
        ax.text(x,15,f'{k+1}')
    if recursion>0:
        filename = filename.replace('.txt','-edge.txt')
    plt.savefig(filename.replace('.txt', '.pdf'))

In [ ]:
def drawBand(filename:str):
    """
    drawing band structure
    """
    ax=plt.subplot()
    ax.set_aspect('equal')
    ax.set_xlim(0,1)
    ax.set_ylim(0,1)
    ax.set_xlabel('$x_n$')
    ax.set_ylabel('$x_{n+1}$')

    data = DataFile(filename)
    l = data.getParam('lambda')
    outputFilename =f'Band-{l}.pdf'
    xl,yl = data.readData()
    ax.plot(xl,yl)

    n = 512
    xl = [k/n for k in range(n)]
    yl = [logistic(x,l,1) for x in xl]
    ax.plot(xl,yl,c='black')
    ax.plot([0,1],[0,1],c='black')
    ax.text(.1,.1,f'$\\lambda={l}$')

    #band structure
    ss = [logistic(.5,l,k) for k in range(1,5)]
    ss.insert(3,.5)
    print(ss)
    #horizontal
    pairs=[(0,2),(1,3),(3,4),(2,4)]
    colors=['orange','red','green','blue']
    patches=[]
    romanN=['I','II','III','IV','V','VI','VII','VIII']
    for k,p in enumerate(pairs):
        w = abs(ss[p[0]]-ss[p[1]])
        x = min(ss[p[0]],ss[p[1]])
        patches.append(pts.Rectangle((x,0),w,.025,color=colors[k]))
        xm = (ss[p[0]]+ss[p[1]])/2
        ax.text(xm,0.05,romanN[k],horizontalalignment='center')
    for v in ss:
        y = logistic(v,l,1)
        ax.plot([v,v],[0,y],c='black')
    for p in patches:
        ax.add_patch(p)
    #vertical
    patches=[]
    for k,q in enumerate(pairs):
        p = (logistic(ss[q[0]],l,1),logistic(ss[q[1]],l,1))
        w = abs(p[0]-p[1])
        x = min(p[0],p[1])
        if k==1:
            patches.append(pts.Rectangle((0.025,x),.025,w,color=colors[k]))
        else:
            patches.append(pts.Rectangle((0,x),.025,w,color=colors[k]))
        xm = (p[0]+p[1])/2
        ax.text(0.05,xm,romanN[k+4],verticalalignment='center')
    for p in patches:
        ax.add_patch(p)
    for v in ss:
        x = logistic(v,l,1)
        ax.plot([0,v],[x,x],c='black')
    outputFilename = f'Band-{l}.pdf'
    plt.savefig(outputFilename)


In [ ]:
def drawBandTransition(filename:str):
    """
    drawing band structure
    """
    ax = plt.subplot()
    ax.set_ylim(0,1)
    ax.axis('off')
    data = DataFile(filename)
    l = data.getParam('lambda')
    outputFilename = f'Band-{l}-transition.pdf'
    W = 0.025
    #band structure
    ss = [logistic(.5,l,k) for k in range(1,5)]
    ss.insert(3,.5)
#    print(ss)
    #horizontal
    pairs=[(0,2),(1,3),(3,4),(2,4)]
    colors=['orange','red','green','blue']
    romanN=['I','II','III','IV','V','VI','VII','VIII']
    patches=[]
    for k,p in enumerate(pairs):
        w = abs(ss[p[0]]-ss[p[1]])
        x = min(ss[p[0]],ss[p[1]])
        patches.append(pts.Rectangle((0,x),W,w,color=colors[k]))
        xm = (ss[p[0]]+ss[p[1]])/2
        ax.text(0.025,xm,romanN[k],verticalalignment='center')
    for p in patches:
        ax.add_patch(p)
    for v in ss:
        y = logistic(v,l,1)
        ax.plot([W,.5],[v,y],c='black')

    patches=[]
    for k in [0,2,3,1]:
        q = pairs[k]
#    for k,q in enumerate(pairs):
        p = (logistic(ss[q[0]],l,1),logistic(ss[q[1]],l,1))
        w = abs(p[0]-p[1])
        x = min(p[0],p[1])
        if k==1:
            patches.append(pts.Rectangle((W/2+.5,x),W/2,w,color=colors[k]))
        else:
            patches.append(pts.Rectangle((.5,x),W,w,color=colors[k]))
        xm = (p[0]+p[1])/2
        ax.text(.5+W,xm,romanN[k+4],verticalalignment='center')
    for p in patches:
        ax.add_patch(p)

    patches=[]
    for k,p in enumerate(pairs):
        w = abs(ss[p[0]]-ss[p[1]])
        x = min(ss[p[0]],ss[p[1]])
        patches.append(pts.Rectangle((1,x),W,w,color=colors[k]))
        xm = (ss[p[0]]+ss[p[1]])/2
        ax.text(W+1,xm,romanN[k],verticalalignment='center')
    for p in patches:
        ax.add_patch(p)
    for v in ss:
        ax.plot([W+.5,1],[v,v],c='black')
    plt.savefig(outputFilename)


In [ ]:
def drawBandFormation(filename:str):
    """
    drawing band formation
    """
    ax = plt.subplot()
    ax.set_xlim(0, 1)
    ax.set_ylim(-1, 20)
    ax.set_xlabel('$x$')
    ax.set_ylabel('$t$')
    
    data = DataFile(filename)
    l = data.getParam('lambda')
    n = int(data.getParam('n'))

    for k in range(n-1):
        xl, yl = data.readData(y = k + 1)
        ax.scatter(yl, xl, c='blue', s=5)
        
    colors=['black', 'red', 'cyan', 'green']
    for k in range(4):
        xl, yl = data.readData(y = k + 1)
        ax.scatter(yl, xl, c=colors[k], marker='s', s=50)
    
    x = logistic(.5, l, 1)
    for k in range(4):
        ax.plot([x,x], [-1,n], c='black')
        x = logistic(x, l, 1)
    outputFilename = filename.replace('.txt', '.pdf')
    plt.savefig(outputFilename)

In [ ]:
def drawMap(l:float, period = 1):
    ax = plt.subplot()
    ax.set_xlim(0, 1)
    ax.set_ylim(0, 1)
    ax.set_xlabel('$x_n$')
    ax.set_ylabel('$x_{n+1}$')
    ax.set_aspect('equal')
    #drawing logistic map
    n = 512
    xl = [k/n for k in range(n)]
    yl = [logistic(x, l, period) for x in xl]
    ax.plot(xl, yl, c='black')
    ax.text(.6,.6,f'$\lambda={l}$')
    plt.savefig(f'LogisticMap-{l}.pdf')


In [ ]:
#filename = 'Logistic-0.957-from-0.1.txt'
#filename = 'Logistic-0.5-from-0.8.txt'
#drawTrajectory(filename)
#filename = 'Logistic-0.957-from-0.45-magnify.txt'
#drawTrajectoryMag(filename, period=3)
#filename='Logistic-0.9-from-0.1.txt'
#drawBand(filename)
#drawBandTransition(filename)
#filename='TimeSequence-0.9.txt'
#drawTimeSequence(filename,0,60)
#filename = 'OverAllView-0.95-1.0.txt'
#overAllView(filename,0.95)
#filename='dist-0.9-0.6.txt'
#drawDist(filename,30,6)
#filename = 'BandFormation-0.9.txt'
#drawBandFormation(filename)
#drawMap(.6)